<a href="https://colab.research.google.com/github/pumkinni/book_study/blob/master/%ED%98%BC%EA%B3%B5_%EB%A8%B8_%EB%94%A5_5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블
> 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 배웁니다.

> 종류
- 랜덤 포레스트 : 부트스트랩 샘플 사용, 대표 앙상블 학습 알고리즘
- 엑스트라 트리 : 결정 트리의 노들르 랜덤하게 분할
- 그레이디언트 부스팅 : 이진 트리의 손실을 보완하는 식으로 얕은 결정 트리를 연속하여 추가함
- 히스토그램 기반 그레이티언트 부스팅 : 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄

#### 정형 데이터와 비정형 데이터
- 정형 데이터
> 어떤 구조로 되어있어 CSV나 데이터베이스, 혹은 엑셀에 저장하기 쉬운 데이터

  > EX) 길이, 높이, 무게
  > 가장 뛰어난 성과를 내는 알고리즘 : 앙상블 학습
- 비정형 데이터
> 데이터베이스나 엑셀로 표현하기 어려운 데이터
  
  >EX) 텍스트 데이터(책의 글), 사진, 디지털 음악
  
  > 잘 맞는 알고리즘 : 신경망 알고리즘

#### 랜덤 포레스트
- 앙상블 학습의 대표 주자
- 결정 트리를 랜덤하게 만들어 결정트리의 숲을 만든 후 각 결정 트리의 예측을 사용해 최종 예측을 만듬
- 부트스트랩 샘플 : 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 만든 훈련 데이터(데이터 세트에서 중복을 허용하여 데이터를 샘플링)
- RandomForestClassifier은 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 랜덤하게 선택하여 사용
- RandomForestRegressor은 전체 특성을 사용
-기본적으로 100개의 결정 트리를 이런 방식으로 훈련 후
- 분류일 경우각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
- 회귀의 경우 단순히 각 트리의 예측을 평균

#### 분류와 회귀 다시 살펴보기
- 분류 : 샘플을 몇 개의 클래스 중 하나로 분류하는 문제
- 회귀 : 임의의 어떤 숫자를 예측하는 문제

In [ ]:
# 데이터 불러오기
import pandas as pd
import numpy as np
wine = pd.read_csv('https://bit.ly/wine_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 테스트 데이터와 훈련 데이터 구분
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size  = 0.2, random_state = 42)

In [ ]:
from pandas.core.common import random_state
# 교차검증
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9973541965122431, 0.8905151032797809)

In [ ]:
# 결정트리와 비교했을 때 당도의 특성 중요도가 줄었다. -> 특성의 일부를 랜덤하게 선택하여 훈련했기 때문 -> 과대적합을 줄이고 일반화 성능이 높아졌다.
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [ ]:
# OOB(out of bag) 샘플을 사용하여 결정트리 평가 (검증 세트 역할)
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
rf.oob_score_

0.8934000384837406

#### 엑스트라 트리
- 기본적으로 100개의 결정 트리를 훈련
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용
- 부트스트랩 샘플을 사용하지 않음(결정트리를 만들 때 전체 훈련 세트를 사용)
- 특성을 무작위로 분할하여 성능은 낮아지지만 과대적합을 막고 검증 세트의 점수를 높인다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs = -1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9974503966084433, 0.8887848893166506)

In [ ]:
# 특성 중요도 확인
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

#### 그레이디언트 부스팅
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
-GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용
-과대적합에 강하고 일반적으로 높은 일반화 성능
-경사 하강법을 사용하여 트리를 앙상블에 추가(결정 트리를 계속 추가하며 가장 낮을 곳을 찾아 이동)
- 분류 : 로지스틱 손실 함수, 회귀 : 평균 제곱 오차 함수

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score= True, n_jobs = -1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.8881086892152563, 0.8720430147331015)

In [ ]:
# 학습률, 트리 개수 증가
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9464595437171814, 0.8780082549788999)

In [ ]:
# 특성 중요도(랜덤 포레스트보다 당도에 더 집중)
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15872278, 0.68010884, 0.16116839])

#### 히스토그램 기반 그레이디언트 부스팅
- 정형데이터 머신러닝 알고리즘 중 가장 인기가 높음
- 입력 특성을 256개의 구간으로 나누고 하나를 누락된 값을 위해 사용(누락된 특성이 있어도 전처리 불필요)
- HistGradientBoostingClassifier은 max_iter(부스팅 반복 횟수 지정)로 트리의 개수를 지정

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9321723946453317, 0.8801241948619236)

In [ ]:
# permutation_imortance() - 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산
# 특성 중요도(importances), 중요도 평균(importances_mean), 중요도 표준 편차(importances_std)를 담고 있음
# n_repeats - 랜덤하게 섞을 횟수를 지정
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result.importances_mean

array([0.08876275, 0.23438522, 0.08027708])

In [ ]:
# 테스트 세트에서 특성 중요도 확인
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리
- XGBoost
- LightGBM

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.8824322471423747, 0.8726214185237284)

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs = -1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9338079582727165, 0.8789710890649293)